In [4]:
# %% [markdown]
# Jupyter-ready multi-agent runner with structured stock output

# %%
import os
import asyncio
from dotenv import load_dotenv
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model
from langgraph_supervisor import create_supervisor
from langchain_core.messages import convert_to_messages

load_dotenv()

def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return
    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)

def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        if len(ns) == 0:
            return
        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label
        print(update_label + "\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]
        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

async def run_agent(query: str):
    """
    Runs the Bright Data MCP-powered multi-agent supervisor for NSE stock analysis.
    Produces structured output per stock with sections:
    Technical, Risk Level, Time Horizon, Volume & Sentiment, Valuation, Event Triggers.
    """

    # --- MCP client (Bright Data) ---
    client = MultiServerMCPClient(
        {
            "bright_data": {
                "command": "npx",
                "args": ["@brightdata/mcp"],
                "env": {
                    "API_TOKEN": os.getenv("BRIGHT_DATA_API_TOKEN"),
                    "WEB_UNLOCKER_ZONE": os.getenv("WEB_UNLOCKER_ZONE", "unblocker"),
                    "BROWSER_ZONE": os.getenv("BROWSER_ZONE", "scraping_browser"),
                },
                "transport": "stdio",
            },
        }
    )
    tools = await client.get_tools()

    # --- Base model ---
    # Keep consistent everywhere to avoid spawning multiple clients
    model = init_chat_model(model="openai:gpt-4-turbo", api_key=os.getenv("OPENAI_API_KEY"))

    # --- Agents ---
    stock_finder_agent = create_react_agent(
        model,
        tools,
        prompt=(
            "You are a stock research analyst specializing in the Indian Stock Market (NSE). "
            "Select EXACTLY 2 actively traded NSE-listed stocks for short-term trading (buy/sell). "
            "Use recent performance, news buzz, volume, or technical strength. Avoid penny/illiquid stocks. "
            "Output ONLY a compact list with: Stock Name and NSE ticker (e.g., TCS, RELIANCE) and 1-line rationale."
        ),
        name="stock_finder_agent",
    )

    market_data_agent = create_react_agent(
        model,
        tools,
        prompt=(
            "You are a market data analyst for NSE stocks. Given a list of NSE tickers (e.g., RELIANCE, INFY), "
            "gather the following for EACH stock (currency INR):\n"
            "- Current price\n- Previous close\n- Today's volume\n- 7-day and 30-day price trend (up/down/flat with % approx)\n"
            "- Technical indicators: RSI(14), 50/200-day moving averages level (above/below), MACD signal (bullish/bearish/flat)\n"
            "- Any notable SPIKES in volume or volatility (mention ratio vs 30-day average if possible)\n"
            "Return data in a concise, structured bullet list per stock. If data is unavailable, write 'N/A'."
        ),
        name="market_data_agent",
    )

    news_analyst_agent = create_react_agent(
        model,
        tools,
        prompt=(
            "You are a financial news analyst for NSE stocks. For each given ticker/name, in the PAST 3–5 DAYS:\n"
            "- Find the most relevant headlines\n- Summarize key updates in 2–4 bullets\n"
            "- Label each stock's overall news sentiment: Positive / Negative / Neutral\n"
            "- Briefly note the short-term price impact (bullish/bearish/unclear)\n"
            "Keep it concise, factual, and clearly attributed per stock. If no fresh news, state 'No major updates'."
        ),
        name="news_analyst_agent",
    )

    # This agent NOW produces the FINAL, STRUCTURED WRITE-UP you requested.
    price_recommender_agent = create_react_agent(
        model,
        tools,
        prompt=(
            "You are a trading strategy advisor for Indian NSE stocks. You are given:\n"
            "- Market data (price, volume, trends, RSI/MAs/MACD)\n"
            "- News summaries and sentiment\n\n"
            "For EACH stock, produce a concise analyst note in EXACTLY this format:\n\n"
            "{Stock Name} ({Ticker})\n\n"
            "Recommendation: <Buy|Sell|Hold>\n"
            "Target Price: INR <number>  # if Hold, you may omit or give a range\n"
            "Reason: <1–2 lines summarizing core rationale>\n\n"
            "Technical: <50/200-day MA position, RSI number + state (overbought/oversold/neutral), MACD signal>\n"
            "Risk Level: <Low|Medium|High> (1 short phrase on why)\n"
            "Time Horizon: <e.g., 3–6 months>\n"
            "Volume & Sentiment: <volume vs avg, overall sentiment>\n"
            "Valuation: <P/E vs sector avg if available, dividend yield>\n"
            "Event Triggers: <upcoming earnings, policy, sector catalysts>\n\n"
            "RULES:\n"
            "- Currency must be INR.\n"
            "- If a field is unavailable, write 'N/A'.\n"
            "- Keep each section to a single line.\n"
            "- Be practical for the NEXT trading day; keep target sensible.\n"
        ),
        name="price_recommender_agent",
    )

    # --- Supervisor ---
    supervisor = create_supervisor(
        model=model,  # reuse same model
        agents=[stock_finder_agent, market_data_agent, news_analyst_agent, price_recommender_agent],
        prompt=(
            "You are a supervisor managing four agents:\n"
            "- stock_finder_agent → pick EXACTLY 2 promising NSE stocks\n"
            "- market_data_agent → fetch current market/technical data\n"
            "- news_analyst_agent → summarize very recent news & sentiment\n"
            "- price_recommender_agent → produce FINAL structured analyst notes as per template\n"
            "Assign work to ONE agent at a time (no parallel calls). Do NOT do work yourself. "
            "Finish end-to-end and return only the final structured notes."
        ),
        add_handoff_back_messages=True,
        output_mode="full_history",
    ).compile()

    # --- Stream results in notebook ---
    final_chunk = None
    async for chunk in supervisor.astream(
        {"messages": [{"role": "user", "content": query}]}
    ):
        pretty_print_messages(chunk, last_message=True)
        final_chunk = chunk

    # Return the final supervisor messages if needed
    if final_chunk and "supervisor" in final_chunk and "messages" in final_chunk["supervisor"]:
        return final_chunk["supervisor"]["messages"]
    return None




In [6]:
await run_agent("Give me good stock recommendation from NSE for short term trading")

Update from node supervisor:

================================= Tool Message =================================
Name: transfer_to_stock_finder_agent

Successfully transferred to stock_finder_agent


Update from node stock_finder_agent:

================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:

================================= Tool Message =================================
Name: transfer_to_market_data_agent

Successfully transferred to market_data_agent


Update from node market_data_agent:

================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:

================================= Tool Message =================================
Name: transfer_to_news_analyst_agent

Successfully transferred to news_analyst_agent


Update fr

[HumanMessage(content='Give me good stock recommendation from NSE for short term trading', additional_kwargs={}, response_metadata={}, id='8f1ce813-051e-4e97-aff2-976e7dfb324c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ilMUJR51P1qKa5izcefLWclE', 'function': {'arguments': '{}', 'name': 'transfer_to_stock_finder_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 228, 'total_tokens': 241, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_de235176ee', 'id': 'chatcmpl-C7GqMGjgsz7divH7jGbUG8ZUqHqud', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--927ae641-93b3-453e-b979-9ef27965409b-0', tool_calls=[{'name': 'trans

In [7]:
await run_agent("What recent news affected Infosys?")

Update from node supervisor:

================================= Tool Message =================================
Name: transfer_to_stock_finder_agent

Successfully transferred to stock_finder_agent


Update from node stock_finder_agent:

================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:

================================= Tool Message =================================
Name: transfer_to_news_analyst_agent

Successfully transferred to news_analyst_agent


Update from node news_analyst_agent:

================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:

================================= Tool Message =================================
Name: transfer_to_price_recommender_agent

Successfully transferred to price_recommender_agen

[HumanMessage(content='What recent news affected Infosys?', additional_kwargs={}, response_metadata={}, id='d08c8063-bc01-44b9-ab8d-433479c42410'),
 AIMessage(content="I'll first need to select specific stocks to focus on. To do this, I'll engage the stock_finder_agent to choose two promising stocks from the NSE. I'll follow up with relevant information regarding Infosys if it is selected.\n\nLet me get the stocks first.", additional_kwargs={'tool_calls': [{'id': 'call_gOw8naJhfS6G07GSdfMbsSQK', 'function': {'arguments': '{}', 'name': 'transfer_to_stock_finder_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 223, 'total_tokens': 292, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_de235176e